In [1]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)

In [2]:
from quanta.clients.yfinance import YahooFinanceClient
from datetime import datetime, timedelta

n=365
from_date = datetime.now() - timedelta(days=n)
to_date = datetime.now() - timedelta(days=1)

yh = YahooFinanceClient()
df_init = yh.get_price(
    "^SPMCBRT",
    from_date=from_date.strftime("%Y-%m-%d"),
    to_date=to_date.strftime("%Y-%m-%d"),
    interval="1h", 
    postclean=True
)

df_init.to_pandas()

,timestamp,datetime,open,high,low,close,volume
0,1731681000,2024-11-15 15:30:00,1228.033203,1233.583252,1224.327515,1227.578613,0
1,1731684600,2024-11-15 16:30:00,1227.338379,1235.298828,1223.589844,1230.812500,0
2,1731688200,2024-11-15 17:30:00,1230.623779,1231.961914,1216.470093,1216.658813,0
3,1731691800,2024-11-15 18:30:00,1216.804565,1217.765259,1211.194580,1212.961670,0
4,1731695400,2024-11-15 19:30:00,1212.961670,1215.191895,1209.316040,1212.670044,0
...,...,...,...,...,...,...,...
1486,1763047800,2025-11-13 16:30:00,1224.808960,1225.718140,1219.977051,1221.038086,0
1487,1763051400,2025-11-13 17:30:00,1222.213867,1224.721436,1219.909058,1221.460449,0
1488,1763055000,2025-11-13 18:30:00,1221.460449,1222.391113,1218.838257,1219.657837,0
1489,1763058600,2025-11-13 19:30:00,1219.657837,1219.909058,1215.361328,1219.342529,0


In [3]:
# import strategy :
from quanta.ta_clients.optimization_strategy import StrategyClient
strat = StrategyClient()

/opt/anaconda3/envs/quanta/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
INDICATOR_CLASSES = strat.get_seed_indicator_config()
INDICATOR_CLASSES


{'SMA': quanta.utils.ta.SMA,
 'RSI': quanta.utils.ta.RSI,
 'MACD': quanta.utils.ta.MACD,
 'BollingerBands': quanta.utils.ta.BollingerBands,
 'ATR': quanta.utils.ta.ATR,
 'Volatility': quanta.utils.ta.Volatility,
 'EngulfingPattern': quanta.utils.ta.EngulfingPattern,
 'OBV': quanta.utils.ta.OBV,
 'VWAP': quanta.utils.ta.VWAP,
 'ADX': quanta.utils.ta.ADX,
 'Stochastic': quanta.utils.ta.Stochastic,
 'WilliamsR': quanta.utils.ta.WilliamsR,
 'CCI': quanta.utils.ta.CCI}

In [5]:
OPTIMIZATION_CONFIG = strat.get_seed_optimization_config()
OPTIMIZATION_CONFIG

{'strategy_name': 'simple_strategy',
 'indicators': {'SMA_short': {'class': 'SMA',
   'params': {'period': {'type': 'int', 'low': 5, 'high': 50}}},
  'SMA_long': {'class': 'SMA',
   'params': {'period': {'type': 'int', 'low': 15, 'high': 50}}},
  'RSI': {'class': 'RSI',
   'params': {'period': {'type': 'int', 'low': 14, 'high': 21}}},
  'ATR': {'class': 'ATR',
   'params': {'period': {'type': 'int', 'low': 10, 'high': 30}}},
  'BollingerBands': {'class': 'BollingerBands',
   'params': {'period': {'type': 'int', 'low': 15, 'high': 30},
    'std_dev': {'type': 'float', 'low': 2.0, 'high': 3.0}}},
  'MACD': {'class': 'MACD',
   'params': {'fast_period': {'type': 'int', 'low': 8, 'high': 16},
    'slow_period': {'type': 'int', 'low': 20, 'high': 30},
    'signal_period': {'type': 'int', 'low': 7, 'high': 12}}},
  'EngulfingPattern': {'class': 'EngulfingPattern',
   'params': {'separate_columns': {'type': 'bool', 'value': False}}},
  'ADX': {'class': 'ADX',
   'params': {'period': {'type': 

In [6]:
strat.print_strategies()

Available strategies:

  simple_strategy:
    Description: Simple strategy based on SMA crossovers and RSI levels with optional Engulfing pattern confirmation.
    Required indicators: ['SMA_short', 'SMA_long', 'RSI', 'MACD', 'ATR', 'BollingerBands']
    Optional indicators: ['ADX', 'CCI', 'EngulfingPattern', 'Stochastic', 'WilliamsR']
    Signal parameters: ['use_engulfing', 'macd_hist_buy_threshold', 'macd_hist_sell_threshold', 'adx_threshold']


In [7]:
STRATEGY = strat.get_strategy_fct("simple_strategy")

In [8]:
# test module OptimizationClient : 
from quanta.ta_clients.optimization import OptimizationClient
opt = OptimizationClient(optimization_config=OPTIMIZATION_CONFIG)


ℹ️  Ignored indicators (not used by 'simple_strategy'): {'CCI', 'Stochastic', 'WilliamsR', 'ADX'}
✓ Strategy 'simple_strategy' initialized with indicators: ['SMA_short', 'SMA_long', 'RSI', 'MACD', 'ATR', 'BollingerBands', 'EngulfingPattern']


In [9]:
opt.optimize(
    df = df_init, 
    backtest_func=STRATEGY, 
    verbose=False)

[I 2025-11-15 21:03:14,508] A new study created in memory with name: trading_strategy_optimization


Best trial: 759. Best value: 15.0048: 100%|██████████| 1000/1000 [00:25<00:00, 39.90it/s, 25.06/3600 seconds]


In [10]:
opt.analyze_parameter_importance()
useless_params = opt.get_useless_parameters(threshold=0.01)
opt.plot_importance()  # Graphique interactif


PARAMETER IMPORTANCE (FANOVA)
ℹ️  Number of omitted trials: 96
ATR_period                    : 0.2908 ✓ Useful
RSI_period                    : 0.2408 ✓ Useful
take_profit                   : 0.1212 ✓ Useful
macd_hist_sell_threshold      : 0.1058 ✓ Useful
BollingerBands_std_dev        : 0.0795 ✓ Useful
stop_loss                     : 0.0498 ✓ Useful
macd_hist_buy_threshold       : 0.0422 ✓ Useful
adx_threshold                 : 0.0282 ✓ Useful
position_size                 : 0.0147 ✓ Useful
MACD_slow_period              : 0.0105 ✓ Useful
SMA_short_period              : 0.0072 ⚠️  USELESS
MACD_fast_period              : 0.0051 ⚠️  USELESS
MACD_signal_period            : 0.0042 ⚠️  USELESS

LOW IMPACT PARAMETERS ANALYSIS

✓ All technical indicators have significant impact

✓ All strategy parameters have significant impact



ℹ️  Number of omitted trials: 96


In [11]:
from quanta.utils.ta import TAClient

indicators = opt.get_best_indicators()
indicators


In [12]:

ta_client = TAClient()
df = ta_client.calculate_indicators(df_init, indicators)

from quanta.utils.trace import Candlesticks, Volume, Line

traces = [
    Candlesticks(),
    Line('hl_avg', name='HL Average', color='purple'),
    Volume()
]


from quanta.clients.chart import ChartClient

chart_client = ChartClient()
chart_client.plot(df, "AAPL", traces=traces, indicators=indicators, theme='professional')

Plotting 1491 bars for AAPL with x_axis_type='row_nb'


In [13]:
trades = opt.get_trades_history(df=df_init)
trades

  Signals generated: 380
  📊 Total signals: 380 (Long: 194, Short: 186)
  💼 Executed trades: 380
  📅 Period: 2024-11-27 16:30:00 → 2025-11-13 20:30:00 (351 days = 0.96 years)
  📊 Trades: 380 | Return: 0.0045 | Sharpe: 0.72 | DD: 6.01%


timestamp,position_number,action,price,quantity_usd,position_size,pnl,cumulative_capital
datetime[μs],i64,str,f64,f64,f64,f64,f64
2024-11-27 20:30:00,0,"""BUY""",1239.671631,0.519128,0.519246,-0.000227,0.999773
2024-11-29 15:30:00,1,"""BUY""",1253.347534,0.522102,0.519246,0.005728,1.0055
2024-11-29 16:30:00,2,"""BUY""",1250.460449,0.521477,0.519246,-0.001196,1.004298
2024-12-02 17:30:00,3,"""BUY""",1228.315796,0.520686,0.519246,-0.001518,1.002773
2024-12-05 19:30:00,4,"""SELL""",1236.206177,0.521194,0.519246,0.000977,1.003753
…,…,…,…,…,…,…,…
2025-11-07 20:30:00,375,"""BUY""",1234.215698,0.519988,0.519246,0.000781,1.00143
2025-11-10 15:30:00,376,"""BUY""",1231.56665,0.519409,0.519246,-0.001114,1.000314
2025-11-10 16:30:00,377,"""BUY""",1228.678223,0.518776,0.519246,-0.001218,0.999095


In [14]:
chart_client = ChartClient()
chart_client.plot(
    df, "AAPL", 
    traces=traces, 
    indicators=indicators, 
    trades_df=trades, 
    theme='professional'
)

Plotting 1491 bars for AAPL with x_axis_type='row_nb'
With 380 trades
Debug: 380 trades after processing
shape: (5, 4)
┌─────────────────────┬─────────┬────────┬─────────────┐
│ timestamp           ┆ x_value ┆ action ┆ price       │
│ ---                 ┆ ---     ┆ ---    ┆ ---         │
│ datetime[μs]        ┆ i64     ┆ str    ┆ f64         │
╞═════════════════════╪═════════╪════════╪═════════════╡
│ 2024-11-27 20:30:00 ┆ 53      ┆ BUY    ┆ 1239.671631 │
│ 2024-11-29 15:30:00 ┆ 54      ┆ BUY    ┆ 1253.347534 │
│ 2024-11-29 16:30:00 ┆ 55      ┆ BUY    ┆ 1250.460449 │
│ 2024-12-02 17:30:00 ┆ 60      ┆ BUY    ┆ 1228.315796 │
│ 2024-12-05 19:30:00 ┆ 80      ┆ SELL   ┆ 1236.206177 │
└─────────────────────┴─────────┴────────┴─────────────┘


In [17]:
chart_client.plot_pnl_distribution(
    trades_df=trades, 
    symbol="AAPL"
)
# Affiche : histogramme PnL + VaR + ES (graphique indépendant)